# Test notebook to exercise the Lasair API

In [1]:
#!/usr/bin/pip3 install lasair --upgrade
!/usr/bin/python3 --version
import sys

Python 3.9.6


Change this to the API token and endpoint you want to test

In [2]:
lsst=False # if you change this restart kernel

if lsst:
    sys.path.append('API_lsst')
    import settings
    endpoint = "https://lasair-lsst.lsst.ac.uk/api"
    oid = 'diaObjectId'
    raname = 'ra'
    decname = 'decl'
else:
    sys.path.append('API_ztf')
    import settings
    endpoint = "https://lasair-ztf.lsst.ac.uk/api"
    oid = 'objectId'
    raname = 'ramean'
    decname = 'decmean'

Fetch your API TOKEN from your Lasair account, top right under "profile"

In [3]:
import json
from lasair import LasairError, lasair_client as lasair
import settings
L = lasair(settings.API_TOKEN, endpoint=endpoint)

/Users/rwillia5/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Cone search

In [4]:
racone  = 279.22
deccone = -47.91
radius = 60 # arcsec
try:
    result = L.cone(racone, deccone, radius, requestType='nearest')
    print(json.dumps(result, indent=2))
except:
    print('nothing found')

{}


## Database query

In [5]:
selected = 'objects.%s, objects.%s, objects.%s' % (oid, raname, decname)
tables = 'objects,sherlock_classifications'
conditions = 'classification="SN"'
results = L.query(selected, tables, conditions, limit = 10)

for row in results:
    objectId = row[oid]
    ra = row[raname]
    dec = row[decname]
    print(objectId, ra, dec)

# Just use this objectId in subsequent calls
objectId = str(objectId)

ZTF17aaaadix 75.12347205 44.35462832500001
ZTF17aaaadqv 71.608009725 44.112756225000005
ZTF17aaaafup 82.7761669 23.7206974
ZTF17aaaajfc 325.6841250888889 47.15376623333333
ZTF17aaaazvg 324.920681575 45.644026475000004
ZTF17aaabcyg 325.2470203 48.2919921
ZTF17aaabjcs 80.7958842 43.3828067
ZTF17aaabjzl 83.2787235 24.4758129
ZTF17aaablao 58.854258099999996 45.597521900000004
ZTF17aaabneq 86.10237208999999 33.67116910000001


## Lightcurves

In [6]:
result = L.object(objectId, lasair_added=False)
print(json.dumps(result, indent=2))

{
  "objectId": "ZTF17aaabneq",
  "candidates": [
    {
      "candid": 612491634315015274,
      "jd": 2458366.9916319,
      "ra": 86.1023997,
      "dec": 33.6711337,
      "fid": 2,
      "nid": 612,
      "magpsf": 18.031299591064453,
      "sigmapsf": 0.0399399995803833,
      "magnr": 20.60099983215332,
      "sigmagnr": 0.03700000047683716,
      "magzpsci": 0.0,
      "isdiffpos": "t",
      "ssdistnr": -999.0,
      "ssnamenr": "null",
      "drb": null
    },
    {
      "candid": 613487444315015128,
      "jd": 2458367.9874421,
      "ra": 86.102387,
      "dec": 33.6711516,
      "fid": 2,
      "nid": 613,
      "magpsf": 17.733400344848633,
      "sigmapsf": 0.038791000843048096,
      "magnr": 20.60099983215332,
      "sigmagnr": 0.03700000047683716,
      "magzpsci": 0.0,
      "isdiffpos": "t",
      "ssdistnr": -999.0,
      "ssnamenr": "null",
      "drb": null
    },
    {
      "candid": 614465354315015161,
      "jd": 2458368.9653588,
      "ra": 86.102401,
     

## Sherlock Object

In [7]:
lite=True
result = L.sherlock_object(objectId, lite=lite)
print(json.dumps(result, indent=2))

{
  "classifications": {
    "ZTF17aaabneq": [
      "SN",
      "The transient is possibly associated with <em>148400861024026001</em>; an r=20.63 mag galaxy found in the PS1 catalogue. It's located 0.06\" N, 0.06\" W from the galaxy centre."
    ]
  },
  "crossmatches": [
    {
      "transient_object_id": "ZTF17aaabneq",
      "association_type": "SN",
      "catalogue_table_name": "PS1",
      "catalogue_object_id": "148400861024026001",
      "catalogue_object_type": "galaxy",
      "raDeg": 86.102393186,
      "decDeg": 33.671151617999996,
      "separationArcsec": 0.089,
      "northSeparationArcsec": "-0.0629352",
      "eastSeparationArcsec": "0.0632045",
      "physical_separation_kpc": null,
      "direct_distance": null,
      "distance": null,
      "z": null,
      "photoZ": null,
      "photoZErr": null,
      "Mag": 20.634,
      "MagFilter": "r",
      "MagErr": 0.004,
      "classificationReliability": 2,
      "major_axis_arcsec": 4.364
    }
  ]
}


## Sherlock Position

In [8]:
result = L.sherlock_position(ra, dec, lite=lite)
print(json.dumps(result, indent=2))

{
  "classifications": {
    "transient_00000": [
      "SN",
      "The transient is possibly associated with <em>148400861024026001</em>; an r=20.63 mag galaxy found in the PS1 catalogue. It's located 0.06\" N, 0.06\" W from the galaxy centre."
    ]
  },
  "crossmatches": [
    {
      "transient_object_id": "transient_00000",
      "association_type": "SN",
      "catalogue_table_name": "PS1",
      "catalogue_object_id": "148400861024026001",
      "catalogue_object_type": "galaxy",
      "raDeg": 86.102393186,
      "decDeg": 33.671151617999996,
      "separationArcsec": 0.089,
      "northSeparationArcsec": "-0.06294",
      "eastSeparationArcsec": "0.06317",
      "physical_separation_kpc": null,
      "direct_distance": null,
      "distance": null,
      "z": null,
      "photoZ": null,
      "photoZErr": null,
      "Mag": 20.634,
      "MagFilter": "r",
      "MagErr": 0.004,
      "classificationReliability": 2,
      "major_axis_arcsec": 4.364
    }
  ]
}


## All object data

In [9]:
result = L.object(objectId, lasair_added=True)
print(json.dumps(result, indent=2))

{
  "objectId": "ZTF17aaabneq",
  "objectData": {
    "ncand": 10,
    "ramean": 86.10237208999999,
    "decmean": 33.67116910000001,
    "glonmean": 175.8246897512245,
    "glatmean": 2.250045897021075,
    "jdmin": 2460666.7019792,
    "jdmax": 2460695.6799306,
    "rasex": "05:44:24.569",
    "decsex": "33:40:16.209",
    "ec_lon": 86.70410620921835,
    "ec_lat": 10.27645968263629,
    "now_mjd": "60699.61",
    "mjdmin_ago": 33.40970680904866,
    "mjdmax_ago": 4.431755409066682,
    "discMjd": 58366.49163189996,
    "discUtc": "2018-09-05 11:47:56",
    "discMag": "18.03\u00b10.04",
    "discFilter": "r",
    "latestMjd": 60695.17993059987,
    "latestUtc": "2025-01-20 04:19:06",
    "latestMag": "19.88\u00b10.14",
    "latestFilter": "g",
    "peakMjd": 60525.49056709977,
    "peakUtc": "2024-08-03 11:46:24",
    "peakMag": "17.13\u00b10.04",
    "peakFilter": "r"
  },
  "candidates": [
    {
      "candid": 2941179934315010127,
      "jd": 2460695.6799306,
      "ra": 86.102381

In [10]:
result = L.object(objectId, lasair_added=False)
print(json.dumps(result, indent=2))

{
  "objectId": "ZTF17aaabneq",
  "candidates": [
    {
      "candid": 612491634315015274,
      "jd": 2458366.9916319,
      "ra": 86.1023997,
      "dec": 33.6711337,
      "fid": 2,
      "nid": 612,
      "magpsf": 18.031299591064453,
      "sigmapsf": 0.0399399995803833,
      "magnr": 20.60099983215332,
      "sigmagnr": 0.03700000047683716,
      "magzpsci": 0.0,
      "isdiffpos": "t",
      "ssdistnr": -999.0,
      "ssnamenr": "null",
      "drb": null
    },
    {
      "candid": 613487444315015128,
      "jd": 2458367.9874421,
      "ra": 86.102387,
      "dec": 33.6711516,
      "fid": 2,
      "nid": 613,
      "magpsf": 17.733400344848633,
      "sigmapsf": 0.038791000843048096,
      "magnr": 20.60099983215332,
      "sigmagnr": 0.03700000047683716,
      "magzpsci": 0.0,
      "isdiffpos": "t",
      "ssdistnr": -999.0,
      "ssnamenr": "null",
      "drb": null
    },
    {
      "candid": 614465354315015161,
      "jd": 2458368.9653588,
      "ra": 86.102401,
     